<a href="https://colab.research.google.com/github/massinoLight/projetSanae/blob/main/bigquery_geotab_bqml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



The kernel uses all the provided ML algorithms:
- KMeans for Geo-Spatial Clustering
- Logistic Regression for Adversarial Validation
- Linear Regression for Prediction Model


## Summary
Features:
- intersectionid + city (*i_id*)
- city, hour, weekend, month
- path, entry_street_name, exit_street_name
- 20 clusters of intersections per city
- Distance to the 5 nearest cluster center
- 8 directions of turn (left, right, centered, uturn, centered-left, ...)
    - Directions, entry- and exit-heading are not embedded directly. Instead they are translated into degrees (e.g. centered = 90deg). 
    - Afterwards the degree features are split additionaly into two features sin(feature_x_deg) and cos(feature_x_deg).
    - Besides the direction of the turn, sin and cos feature are added for entry heading and exit heading 
- number of intersections per cluster
- number of observations per city and hour
- zipcode
- population of zipcode
- population of zipcode per intersection
- intersections per zipcode
- flag, if entry and exit are the same_street
- intersection observation frequency
- approching street length
- approching street length fallback flag, if approching street length was derived from a more general approach due of missing data

Features that didn't improve the model:
- daytime, season (removed from model, since not improving)
- road_type_entry, road_type_exit (removed from model, since not improving)

Train-Valid-Split:
- out of adversarial validation (drop 25 % of not test like data from train) (used by detailed model)
- split by id range (used by general model)

Train-Valid-Split that didn't improved the model:
- remove outliers (target > Q99) from train (removed from model, since not improving)

Model:
- Linear Regression
    - Detailed Model for known path and intersections in train and test
    - General Model for unknown path and intersections



In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [17]:
!pip install kaggle

In [18]:
!cp /root/kaggle.json /content/.kaggle/

In [15]:

!mkdir .kaggle
!touch .kaggle/kaggle.json

api_token = {"username":"amassinnacerddine","key":"5fe036f02abe742412d0d9abdfb2feda"}

import json
import zipfile
import os
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 /content/.kaggle/kaggle.json
!kaggle config path -p /content

mkdir: cannot create directory ‘.kaggle’: File exists
usage: kaggle config [-h] {view,set,unset} ...
kaggle config: error: argument command: invalid choice: 'path' (choose from 'view', 'set', 'unset')


In [19]:
# Replace 'kaggle-competitions-project' with YOUR OWN project id here --  
PROJECT_ID = 'projet-sanae' #
#PROJECT_ID='kaggle-competitions-project'

from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID, location="europe-west1")
dataset = client.create_dataset('bqml_example', exists_ok=True)



import seaborn as sns
import matplotlib.pyplot as plt

# create a reference to our table
table = client.get_table("projet-sanae.BigQueryGeotabIntersectionCongestion.table_train")

# look at five rows from our dataset
client.list_rows(table, max_results=5).to_dataframe()

,RowId,IntersectionId,Latitude,Longitude,EntryStreetName,ExitStreetName,EntryHeading,ExitHeading,Hour,Weekend,Month,Path,TotalTimeStopped_p20,TotalTimeStopped_p40,TotalTimeStopped_p50,TotalTimeStopped_p60,TotalTimeStopped_p80,TimeFromFirstStop_p20,TimeFromFirstStop_p40,TimeFromFirstStop_p50,TimeFromFirstStop_p60,TimeFromFirstStop_p80,DistanceToFirstStop_p20,DistanceToFirstStop_p40,DistanceToFirstStop_p50,DistanceToFirstStop_p60,DistanceToFirstStop_p80,City
0,2083184,33,42.346356,-71.090948,Fenway,Fenway,N,N,9,0,6,Fenway_N_Fenway_N,0.0,0.0,8.0,17.0,27.0,0.0,0.0,15.0,27.0,32.0,0.0,0.0,15.3,37.0,57.9,Boston
1,2083201,33,42.346356,-71.090948,Fenway,Fenway,N,N,15,0,7,Fenway_N_Fenway_N,0.0,10.0,22.0,25.0,54.0,0.0,22.0,33.0,38.0,60.0,0.0,43.2,47.3,51.1,65.7,Boston
2,2083202,33,42.346356,-71.090948,Fenway,Fenway,N,N,16,0,7,Fenway_N_Fenway_N,0.0,0.0,16.0,20.0,44.0,0.0,0.0,29.0,39.0,49.0,0.0,0.0,34.6,52.4,85.7,Boston
3,2083224,33,42.346356,-71.090948,Fenway,Fenway,N,N,13,0,9,Fenway_N_Fenway_N,0.0,10.0,15.0,18.0,21.0,0.0,17.0,21.0,24.0,25.0,0.0,21.0,25.5,29.8,42.0,Boston
4,2083225,33,42.346356,-71.090948,Fenway,Fenway,N,N,16,0,9,Fenway_N_Fenway_N,0.0,0.0,14.0,30.0,49.0,0.0,0.0,23.0,34.0,65.0,0.0,0.0,26.4,52.9,68.9,Boston


In [20]:
%load_ext google.cloud.bigquery

In [21]:

sql = """
    SELECT t.city AS city
      FROM `projet-sanae.BigQueryGeotabIntersectionCongestion.table_train` t
      LIMIT 20"""

job_config = bigquery.QueryJobConfig()
job_config.dry_run = True

query_job = client.query(sql, job_config)

print ("This query will process: {0:.2f} MB".format(
    query_job.total_bytes_processed/1024/1024))


query_job = client.query(sql)
query_job.result()
print ("Total bytes processed (first execution): {0:.2f} MB".format(
    query_job.total_bytes_processed/1024/1024))


query_job = client.query(sql)
query_job.result()
print ("Total bytes processed (cached): {0:.2f} MB".format(
    query_job.total_bytes_processed/1024/1024))

This query will process: 9.04 MB
Total bytes processed (first execution): 0.00 MB
Total bytes processed (cached): 0.00 MB


In [22]:
def run_sql(sql, dry=False):
    if dry:
        job_config = bigquery.QueryJobConfig()
        job_config.dry_run = dry
        query_job = client.query(sql, job_config)
        print ("This query will process: {0:.2f} MB".format(query_job.total_bytes_processed/1024/1024))
    else: 
        query_job = client.query(sql)
        query_job.result()
        print ("Total bytes processed (cached): {0:.2f} MB".format(query_job.total_bytes_processed/1024/1024))
        

In [23]:
# for documentation set model_changed here globaly, for development set later in the appropriate cells
model_changed = False

There will by 12 Models. One for each percentile (20, 50, 80) per category (TotalTimeStopped, DistanceToFirstStop) per model typ (general or detailed).

In [24]:
mod_names= ['TotalTimeStopped_p20','TotalTimeStopped_p50','TotalTimeStopped_p80',
            'DistanceToFirstStop_p20','DistanceToFirstStop_p50','DistanceToFirstStop_p80']

# general model will look like [x+'_general' for x in mod_names]


In [26]:
%env GCLOUD_PROJECT=PROJECT_ID

env: GCLOUD_PROJECT=PROJECT_ID


In [32]:
%%bigquery --project projet-sanae
SELECT 
  COUNT(*) as total_rows
FROM `projet-sanae.BigQueryGeotabIntersectionCongestion.table_train`

,total_rows
0,856387


Distinct cities

In [35]:
%%bigquery --project projet-sanae city_df
SELECT t.city
  FROM `projet-sanae.BigQueryGeotabIntersectionCongestion.table_train` t
UNION DISTINCT
SELECT t.city
  FROM `projet-sanae.BigQueryGeotabIntersectionCongestion.table_train` t


In [36]:
cities = list(city_df['city'])
cities

['Boston', 'Philadelphia', 'Chicago', 'Atlanta']

# Cluster Intersections per City
Build 20 geo spatial clusters of intersections per city:

In [40]:
#model_changed = False

if model_changed:
    for c in cities:
        sql="""DROP MODEL `bqml_example.model_cluster_"""+c+"""`"""
        client.query(sql)

        print('Dropped',c)
    
    
for c in cities:
    sql="""CREATE MODEL IF NOT EXISTS `bqml_example.model_cluster_"""+c+"""`
    OPTIONS(model_type='kmeans',
            NUM_CLUSTERS = 20) AS
    SELECT
        latitude,
        longitude
    FROM
      `projet-sanae.BigQueryGeotabIntersectionCongestion.table_train` t
    WHERE city = '"""+c+"""'
    UNION DISTINCT
    SELECT
        latitude,
        longitude
    FROM
      `projet-sanae.BigQueryGeotabIntersectionCongestion.table_test` t
    WHERE city = '"""+c+"""'"""

    client.query(sql).result()

    print('Done with',c)

Done with Boston
Done with Philadelphia
Done with Chicago
Done with Atlanta


In [44]:
%%bigquery --project projet-sanae eda_df
WITH city_cluster AS (
    SELECT (SELECT MIN(d.DISTANCE) FROM UNNEST(NEAREST_CENTROIDS_DISTANCE) d) AS dist_to_cluster_center, 
           CONCAT(m.city,"_",CAST(m.CENTROID_ID AS STRING)) AS city_cluster,
           m.* EXCEPT (nearest_centroids_distance, CENTROID_ID) 
      FROM ML.PREDICT(MODEL `bqml_example.model_cluster_Boston`, 
                   (SELECT t.RowId,
                   t.city,
                           t.IntersectionId,
                           t.Latitude,
                           t.Longitude,
                           #t.EntryStreetName,
                           #t.ExitStreetName,
                           t.EntryHeading,
                           t.ExitHeading,
                           t.Hour,
                           t.Weekend,
                           t.Month,
                           #t.Path,
                           t.TotalTimeStopped_p20,
                           #t.TotalTimeStopped_p40,
                           t.TotalTimeStopped_p50,
                           #t.TotalTimeStopped_p60,
                           t.TotalTimeStopped_p80,
                           #t.TimeFromFirstStop_p20,
                           #t.TimeFromFirstStop_p40,
                           #t.TimeFromFirstStop_p50,
                           #t.TimeFromFirstStop_p60,
                           #t.TimeFromFirstStop_p80,
                           t.DistanceToFirstStop_p20,
                           #t.DistanceToFirstStop_p40,
                           t.DistanceToFirstStop_p50,
                           #t.DistanceToFirstStop_p60,
                           t.DistanceToFirstStop_p80,
                           'TRAIN' AS source
                     FROM `projet-sanae.BigQueryGeotabIntersectionCongestion.table_train` t
                    WHERE city = 'Boston' 
                    #  AND rowid in(2209678,2209692)
                    UNION ALL
                    SELECT t.RowId,
                    t.city,
                           t.IntersectionId,
                           t.Latitude,
                           t.Longitude,
                           #t.EntryStreetName,
                           #t.ExitStreetName,
                           t.EntryHeading,
                           t.ExitHeading,
                           t.Hour,
                           t.Weekend,
                           t.Month,
                           #t.Path,
                           null as TotalTimeStopped_p20,
                           #null as TotalTimeStopped_p40,
                           null as TotalTimeStopped_p50,
                           #null as TotalTimeStopped_p60,
                           null as TotalTimeStopped_p80,
                           #null as TimeFromFirstStop_p20,
                           #null as TimeFromFirstStop_p40,
                           #null as TimeFromFirstStop_p50,
                           #null as TimeFromFirstStop_p60,
                           #null as TimeFromFirstStop_p80,
                           null as DistanceToFirstStop_p20,
                           #null as DistanceToFirstStop_p40,
                           null as DistanceToFirstStop_p50,
                           #null as DistanceToFirstStop_p60,
                           null as DistanceToFirstStop_p80,
                           'TEST' AS source
                     FROM `projet-sanae.BigQueryGeotabIntersectionCongestion.table_test` t
                    WHERE city = 'Boston' 
                    #  AND rowid in(2209678,2209692)
                    )) m
)
SELECT cc.source,
       cc.city,
       cc.city_cluster, 
       count(1) cnt, 
       avg(cc.dist_to_cluster_center) avg_dist_to_cluster_center, 
       stddev(cc.dist_to_cluster_center) stddev_dist_to_cluster_center, 
       min(cc.dist_to_cluster_center) min_dist_to_cluster_center, 
       max(cc.dist_to_cluster_center) max_dist_to_cluster_center,
       avg(avg(cc.dist_to_cluster_center)*(count(1)-1)) over(partition by cc.city) avg_dist_to_cluster_center_over_city
  FROM city_cluster cc
 GROUP BY cc.source, 
          cc.city, 
          cc.city_cluster;

In [45]:
eda_df.sort_values(by=['source','city_cluster']).head(100)

,source,city,city_cluster,cnt,avg_dist_to_cluster_center,stddev_dist_to_cluster_center,min_dist_to_cluster_center,max_dist_to_cluster_center,avg_dist_to_cluster_center_over_city
22,TEST,Boston,Boston_1,28028,0.232514,0.087005,0.038245,0.414181,4450.787327
25,TEST,Boston,Boston_10,29240,0.285941,0.106315,0.038024,0.464504,4450.787327
0,TEST,Boston,Boston_11,32192,0.426736,0.137220,0.022185,0.695814,4450.787327
5,TEST,Boston,Boston_12,16156,0.380258,0.207233,0.042771,0.914612,4450.787327
6,TEST,Boston,Boston_13,23967,0.175199,0.087805,0.016850,0.395976,4450.787327
19,TEST,Boston,Boston_14,15387,0.473105,0.180840,0.103352,1.202466,4450.787327
28,TEST,Boston,Boston_15,18104,0.347114,0.139943,0.087751,0.643529,4450.787327
27,TEST,Boston,Boston_16,14071,0.211762,0.087128,0.065021,0.354562,4450.787327
21,TEST,Boston,Boston_17,19723,0.335342,0.143018,0.045381,0.643222,4450.787327
24,TEST,Boston,Boston_18,13194,0.265596,0.110775,0.038033,0.476054,4450.787327


In [56]:
def feature_sql(model_name, rowid_split, incl_rowid, tab): 
    
    if incl_rowid:
        rowid = "t.RowId,"
    else:
        rowid = ""
    
    if tab == 'table_test':
        label = ""
    elif model_name == 'ALL':
        label = """t.TotalTimeStopped_p20,
                t.TotalTimeStopped_p50,
                t.TotalTimeStopped_p80,
                t.DistanceToFirstStop_p20,
                t.DistanceToFirstStop_p50,
                t.DistanceToFirstStop_p80,"""
    else:
        label = """t."""+model_name+""" as label,"""
    
    sql = ""
    
    for c in cities:
        features = """SELECT """+label+"""
                             """+rowid+"""
                             t.city,
                             t.EntryHeading,
                             t.ExitHeading,
                             t.Hour,
                             t.Weekend,
                             t.Month,
                             t.Latitude,
                             t.Longitude,
                             case 
                                 when t.entryheading = t.exitheading THEN
                                  "C"
                                 when ("N" in (t.entryheading, t.exitheading) and "S" in (t.entryheading, t.exitheading)) 
                                      OR 
                                      ("E" in (t.entryheading, t.exitheading) and "W" in (t.entryheading, t.exitheading)) 
                                      OR 
                                      ("NE" in (t.entryheading, t.exitheading) and "SW" in (t.entryheading, t.exitheading))  
                                      OR 
                                      ("SE" in (t.entryheading, t.exitheading) and "NW" in (t.entryheading, t.exitheading)) 
                                 THEN
                                  "U" 
                                 when (t.entryheading="N" and t.exitheading = "W") 
                                      OR(t.entryheading="NW" and t.exitheading = "SW") 
                                      OR(t.entryheading="W" and t.exitheading = "S") 
                                      OR(t.entryheading="SW" and t.exitheading = "SE") 
                                      OR(t.entryheading="S" and t.exitheading = "E") 
                                      OR(t.entryheading="SE" and t.exitheading = "NE") 
                                      OR(t.entryheading="E" and t.exitheading = "N") 
                                      OR(t.entryheading="NE" and t.exitheading = "NW") 
                                 THEN
                                  "L" 
                                 when (t.entryheading="N" and t.exitheading = "E") 
                                      OR(t.entryheading="NW" and t.exitheading = "NE") 
                                      OR(t.entryheading="W" and t.exitheading = "N") 
                                      OR(t.entryheading="SW" and t.exitheading = "NW") 
                                      OR(t.entryheading="S" and t.exitheading = "W") 
                                      OR(t.entryheading="SE" and t.exitheading = "SW") 
                                      OR(t.entryheading="E" and t.exitheading = "S") 
                                      OR(t.entryheading="NE" and t.exitheading = "SE") 
                                 THEN
                                  "R" 
                                 when (t.entryheading="N" and t.exitheading = "NW") 
                                      OR(t.entryheading="NW" and t.exitheading = "W") 
                                      OR(t.entryheading="W" and t.exitheading = "SW") 
                                      OR(t.entryheading="SW" and t.exitheading = "S") 
                                      OR(t.entryheading="S" and t.exitheading = "SE") 
                                      OR(t.entryheading="SE" and t.exitheading = "E") 
                                      OR(t.entryheading="E" and t.exitheading = "NE") 
                                      OR(t.entryheading="NE" and t.exitheading = "N") 
                                 THEN
                                  "CL" 
                                 when (t.entryheading="N" and t.exitheading = "NE") 
                                      OR(t.entryheading="NW" and t.exitheading = "N") 
                                      OR(t.entryheading="W" and t.exitheading = "NW") 
                                      OR(t.entryheading="SW" and t.exitheading = "W") 
                                      OR(t.entryheading="S" and t.exitheading = "SW") 
                                      OR(t.entryheading="SE" and t.exitheading = "S") 
                                      OR(t.entryheading="E" and t.exitheading = "SE") 
                                      OR(t.entryheading="NE" and t.exitheading = "E") 
                                 THEN
                                  "CR" 
                                 when (t.entryheading="N" and t.exitheading = "SW") 
                                      OR(t.entryheading="NW" and t.exitheading = "S") 
                                      OR(t.entryheading="W" and t.exitheading = "SE") 
                                      OR(t.entryheading="SW" and t.exitheading = "E") 
                                      OR(t.entryheading="S" and t.exitheading = "NE") 
                                      OR(t.entryheading="SE" and t.exitheading = "N") 
                                      OR(t.entryheading="E" and t.exitheading = "NW") 
                                      OR(t.entryheading="NE" and t.exitheading = "W") 
                                 THEN
                                  "UL" 
                                 when (t.entryheading="N" and t.exitheading = "SE") 
                                      OR(t.entryheading="NW" and t.exitheading = "E") 
                                      OR(t.entryheading="W" and t.exitheading = "NE") 
                                      OR(t.entryheading="SW" and t.exitheading = "N") 
                                      OR(t.entryheading="S" and t.exitheading = "NW") 
                                      OR(t.entryheading="SE" and t.exitheading = "W") 
                                      OR(t.entryheading="E" and t.exitheading = "SW") 
                                      OR(t.entryheading="NE" and t.exitheading = "S") 
                                 THEN
                                  "UR" 
                               else null end direction
                       FROM `projet-sanae.BigQueryGeotabIntersectionCongestion."""+tab+"""` t
                      WHERE city = '"""+c+"""' 
                       AND rowid """+rowid_split
                            
        sql += """
               SELECT (SELECT MIN(d.DISTANCE) FROM UNNEST(NEAREST_CENTROIDS_DISTANCE) d) AS dist_to_cluster_center, 
                      CONCAT(m.city,"_",CAST(m.CENTROID_ID AS STRING)) AS city_cluster,
                      m.* EXCEPT (nearest_centroids_distance, CENTROID_ID,Latitude,Longitude) 
                 FROM ML.PREDICT(MODEL `bqml_example.model_cluster_"""+c+"""`, 
                              ("""+features+""")) m
               UNION ALL"""
        
        
    return sql[:-len("UNION ALL")]

In [59]:
model_changed = False

if model_changed:
    sql="DROP TABLE IF EXISTS `bqml_example.city_cluster_train`"
    job_result=client.query(sql).result()
    sql="DROP TABLE IF EXISTS `bqml_example.city_cluster_test`"
    job_result=client.query(sql).result()

    
sql = "CREATE TABLE IF NOT EXISTS `bqml_example.city_cluster_train` as " + feature_sql('ALL','=rowid', True, 'table_train')
job_result=client.query(sql).result()
sql = "CREATE TABLE IF NOT EXISTS `bqml_example.city_cluster_test` as " + feature_sql('ALL','=rowid', True, 'table_test')
job_result=client.query(sql).result()

In [61]:
%%bigquery --project projet-sanae
SELECT t.TotalTimeStopped_p20 as label, cc.* except (rowid, TotalTimeStopped_p20,TotalTimeStopped_p50,TotalTimeStopped_p80,DistanceToFirstStop_p20, DistanceToFirstStop_p50, DistanceToFirstStop_p80) 
  FROM `bqml_example.city_cluster_train` cc,
       `projet-sanae.BigQueryGeotabIntersectionCongestion.table_train` t
 WHERE cc.rowid = t.rowid
 LIMIT 20;

,label,dist_to_cluster_center,city_cluster,city,EntryHeading,ExitHeading,Hour,Weekend,Month,direction
0,0.0,0.232417,Chicago_9,Chicago,E,E,10,0,6,C
1,30.0,0.232417,Chicago_9,Chicago,E,E,10,0,7,C
2,0.0,0.234208,Chicago_15,Chicago,E,E,9,0,9,C
3,0.0,0.234208,Chicago_15,Chicago,E,E,7,0,7,C
4,0.0,0.234208,Chicago_15,Chicago,E,E,17,0,11,C
5,0.0,0.234208,Chicago_15,Chicago,E,E,10,0,6,C
6,0.0,0.234208,Chicago_15,Chicago,E,E,10,0,12,C
7,0.0,0.234208,Chicago_15,Chicago,E,E,9,0,11,C
8,6.0,0.234208,Chicago_15,Chicago,E,E,14,0,9,C
9,0.0,0.234208,Chicago_15,Chicago,E,E,15,0,11,C


In [62]:
%%bigquery --project projet-sanae
CREATE OR REPLACE FUNCTION `bqml_example.direction2degree`(dir string) AS (
 case dir
   when "C" then
    90
   when 'CL' then
    135
   when "L" then
    180
   when 'UL' then
    225
   when "U" then
    270
   when 'UR' then
    315
   when "R" then
    0
   when 'CR' then
    45
    
   when "N" then
    90
   when 'NW' then
    135
   when "W" then
    180
   when 'SW' then
    225
   when "S" then
    270
   when 'SE' then
    315
   when "E" then
    0
   when 'NE' then
    45
 end
);

""


In [63]:
%%time
model_changed = True

if model_changed:
    for mn in mod_names:
        sql="DROP MODEL IF EXISTS `bqml_example.model_"+mn+"`"
        client.query(sql).result()

        print('Drop',mn)

for mn in mod_names:
    
    sql="""
    CREATE MODEL IF NOT EXISTS `bqml_example.model_"""+mn+"""`
    OPTIONS(MODEL_TYPE='linear_reg', 
            L2_REG=0.1,
            LS_INIT_LEARN_RATE=0.4) AS 
    SELECT  t."""+mn+""" as label,
            cc.city_cluster,
            cc.city,
            cc.hour,
            cc.weekend,
            cc.month,
            cc.direction,
            cc.entryheading,
            cc.exitheading,
            round(sin(bqml_example.direction2degree(cc.direction)*ACOS(-1)/180),6) direction_sin,
            round(cos(bqml_example.direction2degree(cc.direction)*ACOS(-1)/180),6) direction_cos,
            round(sin(bqml_example.direction2degree(cc.entryheading)*ACOS(-1)/180),6) entryheading_sin,
            round(cos(bqml_example.direction2degree(cc.entryheading)*ACOS(-1)/180),6) entryheading_cos,
            round(sin(bqml_example.direction2degree(cc.exitheading)*ACOS(-1)/180),6) exitheading_sin,
            round(cos(bqml_example.direction2degree(cc.exitheading)*ACOS(-1)/180),6) exitheading_cos,
            round(cc.dist_to_cluster_center,8) dist_to_cluster_center
      FROM `bqml_example.city_cluster_train` cc,
           `projet-sanae.BigQueryGeotabIntersectionCongestion.table_train` t
     WHERE t.rowid = cc.rowid
       AND cc.rowid < 2600000;
    """

    client.query(sql).result()

    print('Done with',mn)

Drop TotalTimeStopped_p20
Drop TotalTimeStopped_p50
Drop TotalTimeStopped_p80
Drop DistanceToFirstStop_p20
Drop DistanceToFirstStop_p50
Drop DistanceToFirstStop_p80
Done with TotalTimeStopped_p20
Done with TotalTimeStopped_p50
Done with TotalTimeStopped_p80
Done with DistanceToFirstStop_p20
Done with DistanceToFirstStop_p50
Done with DistanceToFirstStop_p80
CPU times: user 2.03 s, sys: 373 ms, total: 2.4 s
Wall time: 6min 6s


# Test-Validation-Split


**Step 2**

Evaluate the model to see if it can distinquish between test or train. If not (e.g. accuracy = 0.5) the data is assumed to be similar.

In [64]:
%%time
%%bigquery --project projet-sanae
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `bqml_example.model_TotalTimeStopped_p20`)
ORDER BY iteration 

CPU times: user 56.5 ms, sys: 2.79 ms, total: 59.2 ms
Wall time: 2.42 s


,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,0,54.263262,48.084477,None,14637


Results of adversarial validation:

* precision: 0.74410540861649255
* recall: 0.99994428055521567
* accuracy: 0.7622297087132579
* f1_score: 0.85326003277995766
* log_loss: 0.47265372942498035
* roc_auc: 0.756451

==> train and test don't look similar

Examin the test-alike probability of the train data:

In [65]:
%%time
%%bigquery --project projet-sanae
SELECT
  *
FROM
  ML.FEATURE_INFO(MODEL `bqml_example.model_TotalTimeStopped_p20`)

CPU times: user 60.9 ms, sys: 2.29 ms, total: 63.2 ms
Wall time: 2.87 s


,input,min,max,mean,median,stddev,category_count,null_count
0,city_cluster,NaN,NaN,NaN,NaN,NaN,75.0,0
1,city,NaN,NaN,NaN,NaN,NaN,4.0,0
2,hour,0.000000,23.000000,12.346750,12.000000,5.941168,NaN,0
3,weekend,0.000000,1.000000,0.258333,0.000000,0.437719,NaN,0
4,month,1.000000,12.000000,9.095590,9.000000,1.994868,NaN,0
5,direction,NaN,NaN,NaN,NaN,NaN,8.0,0
6,entryheading,NaN,NaN,NaN,NaN,NaN,8.0,0
7,exitheading,NaN,NaN,NaN,NaN,NaN,8.0,0
8,direction_sin,-1.000000,1.000000,0.758663,1.000000,0.426707,NaN,0
9,direction_cos,-1.000000,1.000000,-0.003397,0.000000,0.492282,NaN,0


In [66]:
%%bigquery --project projet-sanae
SELECT
  *
FROM
  ML.WEIGHTS(MODEL  `bqml_example.model_TotalTimeStopped_p20`,
    STRUCT(true AS standardize))

,processed_input,weight,category_weights
0,city_cluster,NaN,"[{'category': 'Boston_13', 'weight': -0.837500..."
1,city,NaN,"[{'category': 'Atlanta', 'weight': 0.134419203..."
2,hour,0.209639,[]
3,weekend,-0.254850,[]
4,month,-0.079381,[]
5,direction,NaN,"[{'category': 'CL', 'weight': -0.1150536366041..."
6,entryheading,NaN,"[{'category': 'NW', 'weight': 0.12509190501801..."
7,exitheading,NaN,"[{'category': 'SW', 'weight': 0.04195477262690..."
8,direction_sin,-0.900823,[]
9,direction_cos,-0.744858,[]


In [67]:
sql="""SELECT
          *
        FROM ML.EVALUATE(MODEL `bqml_example.model_TotalTimeStopped_p20`, (
        SELECT  t.TotalTimeStopped_p20 as label, 
                cc.city_cluster,
                cc.city,
                cc.hour,
                cc.weekend,
                cc.month,
                cc.direction,
                cc.entryheading,
                cc.exitheading,
                round(sin(bqml_example.direction2degree(cc.direction)*ACOS(-1)/180),6) direction_sin,
                round(cos(bqml_example.direction2degree(cc.direction)*ACOS(-1)/180),6) direction_cos,
                round(sin(bqml_example.direction2degree(cc.entryheading)*ACOS(-1)/180),6) entryheading_sin,
                round(cos(bqml_example.direction2degree(cc.entryheading)*ACOS(-1)/180),6) entryheading_cos,
                round(sin(bqml_example.direction2degree(cc.exitheading)*ACOS(-1)/180),6) exitheading_sin,
                round(cos(bqml_example.direction2degree(cc.exitheading)*ACOS(-1)/180),6) exitheading_cos,
                round(cc.dist_to_cluster_center,8) dist_to_cluster_center
          FROM `bqml_example.city_cluster_train` cc,
               `projet-sanae.BigQueryGeotabIntersectionCongestion.table_train` t
         WHERE t.rowid = cc.rowid
         AND t.rowid > 2600000))"""

client.query(sql).to_dataframe()

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,2.723995,30.317636,1.188543,1.459602,-0.000726,0.00556


In [ ]:
print(df.shape)
df.head(100)

(11522010, 2)


,TargetId,Target
0,0_0,1.149341
1,1_0,0.273937
2,2_0,0.198945
3,3_0,0.244206
4,4_0,0.163601
...,...,...
95,95_0,8.534094
96,96_0,0.324148
97,97_0,0.369409
98,98_0,2.491887
